Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this notebook, we will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

You have been provided with a small subset of the credit card applications a bank receives. The dataset has been loaded as a Pandas DataFrame for you. You will start from there. 

Summarize the data with describe() on cc_apps and then split the data into cc_apps_train and cc_apps_test. \
Impute the missing values contained in the dataset. \
Preprocess the data by encoding the categorical features. \
Segregate features and labels for training and testing: X_train, y_train, X_test, and y_test. Then rescale the training and testing features contained in X_train and X_test: rescaledX_train and rescaledX_test. \
Train a logistic regression classifier logreg on (rescaledX_train, y_train) and evaluate on (rescaledX_test, y_test). \
Perform hyperparameter tuning with a GridSearchCV object: grid_model. Once the grid-search process in completed, extract the best model and the best performance score yielded from grid_model.

In [45]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

#cc_apps.info()
cc_apps.nunique()
cc_apps.describe()

cc_apps.drop([11, 13], axis=1, inplace=True)
cc_apps.head()

cc_apps_train, cc_apps_test = train_test_split(cc_apps, test_size=0.33, random_state=42)
cc_apps_train.head()

cc_apps_train_nans_replaced = cc_apps_train.replace("?", np.NaN)
cc_apps_test_nans_replaced = cc_apps_test.replace("?", np.NaN)

cc_apps_train_imputed = cc_apps_train_nans_replaced.fillna(cc_apps_train_nans_replaced.mean())
cc_apps_test_imputed = cc_apps_test_nans_replaced.fillna(cc_apps_train_nans_replaced.mean())

for col in cc_apps_train_imputed.columns:
    if cc_apps_train_imputed[col].dtypes == "object":
        # Impute with the most frequent value
        cc_apps_train_imputed = cc_apps_train_imputed.fillna(cc_apps_train_imputed[col].value_counts().index[0])
        cc_apps_test_imputed = cc_apps_test_imputed.fillna(cc_apps_train_imputed[col].value_counts().index[0])
        
cc_apps_train_cat_encoding = pd.get_dummies(cc_apps_train_imputed)
cc_apps_test_cat_encoding = pd.get_dummies(cc_apps_test_imputed)

cc_apps_test_cat_encoding = cc_apps_test_cat_encoding.reindex(columns=cc_apps_train_cat_encoding.columns, fill_value=0)

X_train, y_train = (cc_apps_train_cat_encoding.iloc[:, :-1].values,cc_apps_train_cat_encoding.iloc[:, [-1]].values)
X_test, y_test = (cc_apps_test_cat_encoding.iloc[:, :-1].values, cc_apps_test_cat_encoding.iloc[:, [-1]].values)

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

logreg = LogisticRegression()
logreg.fit(rescaledX_train, y_train)
y_pred = logreg.predict(rescaledX_test)

print(confusion_matrix(y_test, y_pred))

tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]
param_grid = dict(tol=tol, max_iter=max_iter)
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

grid_model_result = grid_model.fit(rescaledX_train, y_train)
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

best_model = grid_model_result.best_estimator_
print("Accuracy of logistic regression classifier: ", best_model.score(rescaledX_test, y_test))

[[103   0]
 [  0 125]]
Best: 1.000000 using {'max_iter': 100, 'tol': 0.01}
Accuracy of logistic regression classifier:  1.0
